In [40]:
import pandas as pd

df = pd.read_json('result.json')
df = pd.json_normalize(df.messages)

### Кто больше всего написал в чате? Топ 10.

In [48]:
df['from'].value_counts().head(10)

rpuropuu [Greeg'O'Rii']    3011
Анатолий Карпов            2076
Max 🍿                      1741
oleg                       1132
esperanto                  1055
Ian Pilé                   1036
Dmitry                      983
Artem Gruzdov               810
Ilya Shishov                786
Yana                        755
Name: from, dtype: int64

### Кто чаще отвечает на чьи сообщения? Топ 10 "отвечателей".

In [49]:
# lets leave columns that are needed for responses statistics calculation
df_response = df[['from_id', 'from','id', 'reply_to_message_id']]
df_response.head()

,from_id,from,id,reply_to_message_id
0,NaN,NaN,1,NaN
1,channel1430200876,Karpov Courses,3,NaN
2,channel1430200876,Karpov Courses,4,NaN
3,NaN,NaN,5,NaN
4,user467542724,Artem Sedlovsky,6,4.0


In [15]:
#Top 10 responders
df_top_response = df_response.loc[df_response['reply_to_message_id'].isnull() == False]
df_top_response = df_top_response.groupby(['from_id']).size().sort_values(ascending=False).rename_axis('from_id').reset_index(name='number_of_replies')
df_top_response.head(10)


,from_id,number_of_replies
0,user448250544,1472
1,user1420037240,1173
2,user1474153825,842
3,user148158343,746
4,user58725603,637
5,user458445864,591
6,user474119900,553
7,user290809359,405
8,channel1173846440,387
9,user539902693,384


In [6]:
#List of top 10 responders (needed further)
top_repliers = df_top_response['from_id'].head(10).tolist()
top_repliers

['user448250544',
 'user1420037240',
 'user1474153825',
 'user148158343',
 'user58725603',
 'user458445864',
 'user474119900',
 'user290809359',
 'channel1173846440',
 'user539902693']

In [76]:
# table with responders and their recipients(whom was replied) 
df_response = df_response[df_response.reply_to_message_id != 'NaN'].merge(df_response, left_on='reply_to_message_id', right_on='id', how='left')
df_response

,from_id_x,from_x,id_x,reply_to_message_id_x,from_id_y,from_y,id_y,reply_to_message_id_y
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
1,channel1430200876,Karpov Courses,3,NaN,NaN,NaN,NaN,NaN
2,channel1430200876,Karpov Courses,4,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
4,user467542724,Artem Sedlovsky,6,4.0,channel1430200876,Karpov Courses,4.0,NaN
...,...,...,...,...,...,...,...,...
37728,channel1430200876,Karpov Courses,43134,NaN,NaN,NaN,NaN,NaN
37729,channel1430200876,Karpov Courses,43135,NaN,NaN,NaN,NaN,NaN
37730,user179404546,андрей тенников,43136,43135.0,channel1430200876,Karpov Courses,43135.0,NaN
37731,user325769634,Roman Bunin,43138,43136.0,user179404546,андрей тенников,43136.0,43135.0


In [80]:
# Top 10 responders to one recipient, count is number of replies 
df_response_info = df_response.rename(columns={"from_id_x": "from_id", "from_x": "from","id_x": "message_id","reply_to_message_id_x": "reply_to_message_id","from_id_y": "to_id", "from_y": "to_name"})
df_response_info = df_response_info[['from_id','from','message_id','reply_to_message_id','to_id', 'to_name']]
df_from_to_repliers = df_response_info.loc[df_response_info['from_id'].isin(top_repliers)]
df_from_to_repliers = df_from_to_repliers.groupby(['from_id','to_id']).size().sort_values(ascending=False).reset_index(name='counts')
df_from_to_repliers.head(10)


,from_id,to_id,counts
0,user448250544,user1420037240,468
1,user1420037240,user448250544,393
2,user1420037240,user458445864,99
3,user448250544,user148158343,87
4,user448250544,user458445864,86
5,user1420037240,user474119900,80
6,user458445864,user448250544,65
7,user458445864,user1420037240,64
8,user474119900,user1420037240,61
9,user448250544,user145764260,60


### Насколько это взаимно? (творческий вопрос, используйте любые метрики)

In [81]:
df_response_info

,from_id,from,message_id,reply_to_message_id,to_id,to_name
0,NaN,NaN,1,NaN,NaN,NaN
1,channel1430200876,Karpov Courses,3,NaN,NaN,NaN
2,channel1430200876,Karpov Courses,4,NaN,NaN,NaN
3,NaN,NaN,5,NaN,NaN,NaN
4,user467542724,Artem Sedlovsky,6,4.0,channel1430200876,Karpov Courses
...,...,...,...,...,...,...
37728,channel1430200876,Karpov Courses,43134,NaN,NaN,NaN
37729,channel1430200876,Karpov Courses,43135,NaN,NaN,NaN
37730,user179404546,андрей тенников,43136,43135.0,channel1430200876,Karpov Courses
37731,user325769634,Roman Bunin,43138,43136.0,user179404546,андрей тенников


In [82]:
#Информация не только о топ 10 'отвечателях' + ответное количество replies 
df_mutual_replies = df_response_info.loc[df_response_info['from_id'].isin(top_repliers)]
df_mutual_replies = df_response_info.loc[df_response_info['to_id'].isin(top_repliers)]
df_mutual_replies = df_mutual_replies.groupby(['from_id','to_id']).size().sort_values(ascending=False).reset_index(name='counts')
df_mutual_replies.head(10)

,from_id,to_id,counts
0,user448250544,user1420037240,468
1,user1420037240,user448250544,393
2,user1420037240,user458445864,99
3,user448250544,user148158343,87
4,user448250544,user458445864,86
5,user1420037240,user474119900,80
6,user458445864,user448250544,65
7,user458445864,user1420037240,64
8,user474119900,user1420037240,61
9,user1474153825,user474119900,55


In [83]:
# reply statistics for specified user
user_id = 'user1474153825'
df_for_user =  df_mutual_replies[(df_mutual_replies['from_id'] == user_id) | (df_mutual_replies['to_id'] == user_id)]
df_for_user


,from_id,to_id,counts
9,user1474153825,user474119900,55
11,user474119900,user1474153825,42
17,user293947789,user1474153825,40
20,user87604320,user1474153825,36
21,user1474153825,user148158343,36
...,...,...,...
1428,user2009464215,user1474153825,1
1431,user420385366,user1474153825,1
1449,user410885384,user1474153825,1
1456,user205154668,user1474153825,1


### В какой час дня исторически больше всего пишут в чат?


In [92]:
df_messages_by_time = df[['id', 'date']]
df_messages_by_time.head()

,id,date
0,1,2021-01-05T16:26:39
1,3,2021-01-05T17:00:17
2,4,2021-01-05T17:00:17
3,5,2021-01-05T18:27:41
4,6,2021-01-05T22:29:20


In [93]:
def get_hour(row):
    return row['date'].split('T')[1][0:2]
df_messages_by_time['hour'] = df_messages_by_time.apply (lambda row: get_hour(row), axis=1)
df_messages_by_time.head()

C:\Users\Acer\AppData\Local\Temp\ipykernel_5672\3793397293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_by_time['hour'] = df_messages_by_time.apply (lambda row: get_hour(row), axis=1)


,id,date,hour
0,1,2021-01-05T16:26:39,16
1,3,2021-01-05T17:00:17,17
2,4,2021-01-05T17:00:17,17
3,5,2021-01-05T18:27:41,18
4,6,2021-01-05T22:29:20,22


In [94]:
df_messages_by_time['hour'].value_counts().head(1)


20    3162
Name: hour, dtype: int64

### В какой день произошло самое большое падение количества сообщений в чате в % к предыдущему дню?

In [95]:
def get_day(row):
    return row['date'].split('T')[0]
df_messages_by_time['day'] = df_messages_by_time.apply (lambda row: get_day(row), axis=1)
df_messages_by_time.head()

C:\Users\Acer\AppData\Local\Temp\ipykernel_5672\2215528191.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messages_by_time['day'] = df_messages_by_time.apply (lambda row: get_day(row), axis=1)


,id,date,hour,day
0,1,2021-01-05T16:26:39,16,2021-01-05
1,3,2021-01-05T17:00:17,17,2021-01-05
2,4,2021-01-05T17:00:17,17,2021-01-05
3,5,2021-01-05T18:27:41,18,2021-01-05
4,6,2021-01-05T22:29:20,22,2021-01-05


In [96]:
# count how many messages were sent each day
df_new = pd.DataFrame({'day_count' : df_messages_by_time.groupby( [ "day"] ).size()}).reset_index()
df_new.head()

,day,day_count
0,2021-01-05,5
1,2021-01-09,4
2,2021-01-10,4
3,2021-01-11,1
4,2021-01-12,1


In [100]:
df_new['%'] = df_new['day_count'].div(df_new['day_count'].shift(1))
df_new.head(50)

,day,day_count,%
0,2021-01-05,5,NaN
1,2021-01-09,4,0.800000
2,2021-01-10,4,1.000000
3,2021-01-11,1,0.250000
4,2021-01-12,1,1.000000
5,2021-01-13,18,18.000000
6,2021-01-14,4,0.222222
7,2021-01-15,3,0.750000
8,2021-01-16,3,1.000000
9,2021-01-17,2,0.666667


In [98]:
# the most decreasing amount of messages day in comparison to previous day. We skip days when there we no messages cau
df_new.min()

day          2021-01-05
day_count             1
%              0.041667
dtype: object

In [ ]:
import requests
import pandas as pd
import lxml

In [ ]:
url = 'https://teaching.healthtech.dtu.dk/22110/index.php/Codon_list'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
df

In [ ]:
df = df.rename(columns={"Amino Acid": "Aminoacid", "DNA codons": "DNA_codons"})
df

In [ ]:
df['DNA_codons'] = df['DNA_codons'].str.strip('()').str.split(',',5,expand=False)
df

In [ ]:
df = df.explode('DNA_codons').reset_index(drop=True)
df

In [ ]:
aminoacid = df.loc[df['DNA_codons'] =='ATC', 'SLC'].iloc[0]
aminoacid

In [ ]:
f = df.loc[df['DNA_codons'] =='ATA', 'SLC']
f
# можно найти индекс и по нему вытащить значение аминокислоты вторым действием

In [ ]:
ind = df.index[df['DNA_codons']=='ATC'].tolist()
ind